# An example of using the Bert model with pytorch
https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial

In [12]:
import pandas as pd
import transformers
from transformers import pipeline
import pandas as pd
import numpy as np

unmasker = pipeline("fill-mask", model='bert-base-cased')
unmasker("Data Science is a [MASK] topic.")

In [2]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
text = "Four Score and Seven Years ago"
encoded_input = tokenizer(text, return_tensors='pt')
print("encoded input")
print(encoded_input)
print("model output")
output = model(**encoded_input)
print(output)

encoded input
{'input_ids': tensor([[  101,  3396, 18417,  1105,  5334,  5848,  2403,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
model output
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0954,  0.0860, -0.1755,  ..., -0.1215,  0.2177, -0.1546],
         [-0.2367, -0.5680, -0.0705,  ...,  0.2298,  0.5074, -0.3337],
         [ 0.1411, -0.6689,  0.3907,  ..., -0.2502,  0.5436, -0.1273],
         ...,
         [ 0.2350,  0.1372,  0.0779,  ...,  0.5715, -0.0502, -0.6023],
         [-0.0924, -0.8698, -0.2965,  ..., -0.4819,  0.1679, -0.3317],
         [ 0.1322, -0.5264, -0.5078,  ...,  0.0271, -0.0999, -0.4042]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-5.4351e-01,  4.3489e-01,  9.9989e-01, -9.9644e-01,  9.6890e-01,
          9.1887e-01,  9.8558e-01, -9.8568e-01, -9.9278e-01, -5.3747e-01,
          9.9226e-01,  9.9943e-01, -9.9700e-01, -9.9982e-01,  8.1915e-01

In [4]:
output.last_hidden_state

tensor([[[ 0.0954,  0.0860, -0.1755,  ..., -0.1215,  0.2177, -0.1546],
         [-0.2367, -0.5680, -0.0705,  ...,  0.2298,  0.5074, -0.3337],
         [ 0.1411, -0.6689,  0.3907,  ..., -0.2502,  0.5436, -0.1273],
         ...,
         [ 0.2350,  0.1372,  0.0779,  ...,  0.5715, -0.0502, -0.6023],
         [-0.0924, -0.8698, -0.2965,  ..., -0.4819,  0.1679, -0.3317],
         [ 0.1322, -0.5264, -0.5078,  ...,  0.0271, -0.0999, -0.4042]]],
       grad_fn=<NativeLayerNormBackward0>)

In [28]:
last_shape = output.last_hidden_state.shape
elements = np.cumproduct(last_shape)
print(np.max(elements))
last_layer_vector = output.last_hidden_state.detach().numpy().reshape(max(elements))
print(last_layer_vector.shape)

6144
(6144,)


In [29]:
np.save("layer.npy", last_layer_vector,allow_pickle=False)

In [30]:
np.load("layer.npy")

array([ 0.09544564,  0.08602602, -0.17547004, ...,  0.02706809,
       -0.09994922, -0.4041806 ], dtype=float32)

In [4]:
from torchvision.models.feature_extraction import create_feature_extractor

In [5]:
model._modules

OrderedDict([('embeddings',
              BertEmbeddings(
                (word_embeddings): Embedding(28996, 768, padding_idx=0)
                (position_embeddings): Embedding(512, 768)
                (token_type_embeddings): Embedding(2, 768)
                (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )),
             ('encoder',
              BertEncoder(
                (layer): ModuleList(
                  (0): BertLayer(
                    (attention): BertAttention(
                      (self): BertSelfAttention(
                        (query): Linear(in_features=768, out_features=768, bias=True)
                        (key): Linear(in_features=768, out_features=768, bias=True)
                        (value): Linear(in_features=768, out_features=768, bias=True)
                        (dropout): Dropout(p=0.1, inplace=False)
                      )
                      (output

In [8]:
return_nodes = {"encoder":"encoder_layer"}

In [11]:
model2 = create_feature_extractor(model, return_nodes=["encoder"])

ValueError: You cannot specify both input_ids and inputs_embeds at the same time